In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re

In [112]:
df = pd.read_csv('./billionaires.csv')

pd.set_option('display.max_columns', None)
df

,name,rank,year,company.founded,company.name,company.relationship,company.sector,company.type,demographics.age,demographics.gender,location.citizenship,location.country code,location.gdp,location.region,wealth.type,wealth.worth in billions,wealth.how.category,wealth.how.from emerging,wealth.how.industry,wealth.how.inherited,wealth.how.was founder,wealth.how.was political
0,Bill Gates,1,1996,1975,Microsoft,founder,Software,new,40,male,United States,USA,8.100000e+12,North America,founder non-finance,18.5,New Sectors,True,Technology-Computer,not inherited,True,True
1,Bill Gates,1,2001,1975,Microsoft,founder,Software,new,45,male,United States,USA,1.060000e+13,North America,founder non-finance,58.7,New Sectors,True,Technology-Computer,not inherited,True,True
2,Bill Gates,1,2014,1975,Microsoft,founder,Software,new,58,male,United States,USA,0.000000e+00,North America,founder non-finance,76.0,New Sectors,True,Technology-Computer,not inherited,True,True
3,Warren Buffett,2,1996,1962,Berkshire Hathaway,founder,Finance,new,65,male,United States,USA,8.100000e+12,North America,founder non-finance,15.0,Traded Sectors,True,Consumer,not inherited,True,True
4,Warren Buffett,2,2001,1962,Berkshire Hathaway,founder,Finance,new,70,male,United States,USA,1.060000e+13,North America,founder non-finance,32.3,Traded Sectors,True,Consumer,not inherited,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,Wu Chung-Yi,1565,2014,1991,Tingyi,investor,beverages and food,new,55,male,Taiwan,Taiwan,0.000000e+00,East Asia,executive,1.0,Traded Sectors,True,Non-consumer industrial,not inherited,True,True
2610,Wu Xiong,1565,2014,1999,Biostime International Holdings,owner,infant formula,new,0,male,China,CHN,0.000000e+00,East Asia,executive,1.0,Traded Sectors,True,Consumer,not inherited,True,True
2611,Yang Keng,1565,2014,0,Blue Ray Corp,chairman,real estate,new,53,male,China,CHN,0.000000e+00,East Asia,self-made finance,1.0,Financial,True,Real Estate,not inherited,True,True
2612,Zdenek Bakala,1565,2014,1994,Patria Finance,founder,coal,new,53,male,Czech Republic,CZE,0.000000e+00,Europe,privatized and resources,1.0,Resource Related,True,Mining and metals,not inherited,True,True


In [113]:
print("The dataset consists of %d features and %d instances." %df.shape)

The dataset consists of 2614 features and 22 instances.


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2614 entries, 0 to 2613
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      2614 non-null   object 
 1   rank                      2614 non-null   int64  
 2   year                      2614 non-null   int64  
 3   company.founded           2614 non-null   int64  
 4   company.name              2576 non-null   object 
 5   company.relationship      2568 non-null   object 
 6   company.sector            2591 non-null   object 
 7   company.type              2578 non-null   object 
 8   demographics.age          2614 non-null   int64  
 9   demographics.gender       2580 non-null   object 
 10  location.citizenship      2614 non-null   object 
 11  location.country code     2614 non-null   object 
 12  location.gdp              2614 non-null   float64
 13  location.region           2614 non-null   object 
 14  wealth.t

# Preprocessing
This includes data cleanse, filling missing data, and removing duplicates.

## Cleaning up features

### General touches

In [115]:
# Removing trailing and leading spaces from names
# Removing all consecutive spaces in the middle of the names
# Removing all extra dots

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].apply(lambda x: x.strip().replace('..', '.').replace('  ', ' ') if isinstance(x, str) else x)

### wealth.how.was political

In [116]:
df['wealth.how.was political'].value_counts()

True    2614
Name: wealth.how.was political, dtype: int64

### wealth.how.was founder

In [117]:
df['wealth.how.was founder'].value_counts()

True    2614
Name: wealth.how.was founder, dtype: int64

### wealth.how.inherited

In [118]:
df['wealth.how.inherited'].value_counts()

not inherited               1688
father                       558
3rd generation               210
4th generation                68
spouse/widow                  59
5th generation or longer      31
Name: wealth.how.inherited, dtype: int64

### wealth.how.industry

In [119]:
df['wealth.how.industry'].value_counts()

Consumer                           471
Retail, Restaurant                 281
Real Estate                        280
Money Management                   249
Media                              219
Technology-Computer                208
Diversified financial              167
Energy                             132
Technology-Medical                 111
Non-consumer industrial            107
Constrution                         97
Mining and metals                   90
Other                               83
Hedge funds                         67
Private equity/leveraged buyout     25
0                                   16
Venture Capital                      8
banking                              1
services                             1
Name: wealth.how.industry, dtype: int64

In [120]:
df['wealth.how.industry'] = df['wealth.how.industry'].replace(
    {
        '0': 'Unspecified',
        'Retail, Restaurant': 'Retail/Restaurant',
        'Technology-Computer': 'Technology/Computer',
        'Technology-Medical': 'Technology/Medical',
    }
)

In [121]:
df['wealth.how.industry'].isna().sum()

1

In [122]:
df['wealth.how.industry'] = df['wealth.how.industry'].fillna('Unspecified')

### wealth.how.from emerging

In [123]:
df['wealth.how.from emerging'].value_counts()

True    2614
Name: wealth.how.from emerging, dtype: int64

### wealth.how.category

In [124]:
df['wealth.how.category'].value_counts()

Financial             800
Non-Traded Sectors    597
Traded Sectors        564
New Sectors           319
Resource Related      245
0                      85
Finance                 1
Trucking                1
energy                  1
Name: wealth.how.category, dtype: int64

In [125]:
df['wealth.how.category'] = df['wealth.how.category'].replace(
    {
        '0': 'Unspecified'
    }
)

In [126]:
df['wealth.how.category'].isna().sum()

1

In [127]:
df['wealth.how.category'] = df['wealth.how.category'].fillna('Unspecified')

### wealth.type

In [128]:
df['wealth.type'].value_counts()

inherited                   953
founder non-finance         713
self-made finance           500
privatized and resources    236
executive                   190
Name: wealth.type, dtype: int64

In [129]:
df['wealth.type'].isna().sum()

22

In [130]:
df['wealth.type'] = df['wealth.type'].fillna('Unspecified')

### location.region

In [131]:
df['location.region'].value_counts()

North America               992
Europe                      698
East Asia                   535
Latin America               182
Middle East/North Africa    117
South Asia                   69
Sub-Saharan Africa           20
0                             1
Name: location.region, dtype: int64

In [132]:
df['location.region'] = df['location.region'].replace(
    {
        '0': 'Unspecified'
    }
)

In [133]:
df['location.region'].isna().sum()

0

### location.country code

In [134]:
country_codes = df['location.country code'].unique()
country_codes.sort()
country_codes

array(['AGO', 'ARE', 'ARG', 'AUS', 'AUT', 'BEL', 'BHR', 'BMU', 'BRA',
       'CAN', 'CHE', 'CHL', 'CHN', 'COL', 'CYP', 'CZE', 'DEN', 'DEU',
       'DNK', 'DZA', 'ECU', 'EGY', 'ESP', 'FIN', 'FRA', 'GBR', 'GEO',
       'GGY', 'GRC', 'HKG', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN',
       'KAZ', 'KNA', 'KOR', 'KWT', 'LBN', 'LIE', 'LTU', 'MAC', 'MAR',
       'MCO', 'MEX', 'MYS', 'NGA', 'NLD', 'NOR', 'NPL', 'NZL', 'OMN',
       'PER', 'PHL', 'POL', 'PRT', 'ROU', 'RUS', 'SAU', 'SGP', 'SWE',
       'SWZ', 'THA', 'TUR', 'TZA', 'Taiwan', 'UGA', 'UKR', 'USA', 'VEN',
       'VNM', 'ZAF'], dtype=object)

In [135]:
df['location.country code'].isna().sum()

0

### location.citizenship

In [136]:
citizenships = df['location.citizenship'].unique()
citizenships.sort()
citizenships

array(['Algeria', 'Angola', 'Argentina', 'Australia', 'Austria',
       'Bahrain', 'Belgium', 'Bermuda', 'Brazil', 'Canada', 'Chile',
       'China', 'Colombia', 'Cyprus', 'Czech Republic', 'Denmark',
       'Ecuador', 'Egypt', 'Finland', 'France', 'Georgia', 'Germany',
       'Greece', 'Guernsey', 'Hong Kong', 'India', 'Indonesia', 'Ireland',
       'Israel', 'Italy', 'Japan', 'Kazakhstan', 'Kuwait', 'Lebanon',
       'Liechtenstein', 'Lithuania', 'Macau', 'Malaysia', 'Mexico',
       'Monaco', 'Morocco', 'Nepal', 'Netherlands', 'New Zealand',
       'Nigeria', 'Norway', 'Oman', 'Peru', 'Philippines', 'Poland',
       'Portugal', 'Romania', 'Russia', 'Saudi Arabia', 'Singapore',
       'South Africa', 'South Korea', 'Spain', 'St. Kitts and Nevis',
       'Swaziland', 'Sweden', 'Switzerland', 'Taiwan', 'Tanzania',
       'Thailand', 'Turkey', 'Uganda', 'Ukraine', 'United Arab Emirates',
       'United Kingdom', 'United States', 'Venezuela', 'Vietnam'],
      dtype=object)

In [137]:
df['location.citizenship'].isna().sum()

0

### demographics.gender

In [138]:
df['demographics.gender'].value_counts()

male              2328
female             249
married couple       3
Name: demographics.gender, dtype: int64

In [139]:
df['demographics.gender'].isna().sum()

34

In [140]:
df['demographics.gender'] = df['demographics.gender'].fillna('Unspecified')

### company.type

In [146]:
company_types = df['company.type'].dropna().unique()
company_types.sort()
company_types

array(['acquired', 'franchise', 'joint venture', 'merger', 'new',
       'new/aquired', 'new/privatized', 'privatized',
       'state owned enterprise', 'subsidiary'], dtype=object)

In [145]:
df['company.type'] = df['company.type'].replace(
    {
        'aquired': 'acquired',
        'franchise rights': 'franchise',
        'neew': 'new',
        'new division': 'new',
        'new, privitization': 'new/privatized',
        'privatization': 'privatized'
    }
)

In [150]:
df['company.type'].isna().sum()

0

In [149]:
df['company.type'] = df['company.type'].fillna('Unspecified')

### company.sector

In [159]:
sectors = df['company.sector'].dropna().unique()
sectors.sort()
sectors

array(['GPS technology', 'HR consulting', 'IT Consulting', 'Star Wars',
       'advertising', 'aerospace and defense', 'agribusiness',
       'agriculteral', 'agricultural products', 'agriculture',
       'aigriculture', 'air compressors', 'aircraft leasing', 'airline',
       'airplanes', 'airport', 'airport maintenance', 'airports',
       'alcohol', 'aluminum', 'aluminum, oil', 'animal feed', 'animation',
       'antennas', 'aplliances', 'apparel', 'apparel and eyewear',
       'apparel retail', 'appliances', 'architectural products',
       'armaments', 'art dealing', 'asset management', 'auto components',
       'auto dealerships', 'auto engines', 'auto glass', 'auto parts',
       'auto repair', 'auto retail', 'auto sales, energy',
       'automobile dealers', 'automobiles', 'automotive',
       'automotive components', 'automotive parts', 'automotives',
       'autos', 'aviation', 'banannas', 'bank', 'banking',
       'banking and insurance', 'banking and real estate',
       'b

In [158]:

df['company.sector'] = df['company.sector'].replace(
    {
        'Banking': 'banking',
        'Communications': 'communications',
        'Fashion': 'fashion',
        'Finance': 'finance',
        'Oil refining': 'oil refining',
        'Software': 'software'
    }
)

### company.relationship

In [65]:
list(df['company.relationship'].unique())

['founder',
 'nan',
 'founder, chairman',
 'relation',
 'ceo',
 'chairman',
 'investor',
 'founder and ceo',
 'founder, ceo',
 'owner',
 'chairman of management committee',
 'founder and chairman',
 'chairman and chief executive officer',
 'general director',
 'executive chairman',
 'chairman, founder',
 'founder, chairman, ceo',
 'former chairman and ceo',
 'relation and chairman',
 'investor, founder',
 'partner',
 'president',
 'investor and ceo',
 'founder ceo owner',
 'chairman, shareholder',
 'vice president',
 'chairman of the board',
 'founder, relation',
 'founder, vice chairman',
 'honorary president for life',
 'former ceo',
 'inherited',
 'vice-chairman',
 'owner and vice chair',
 'founder, president',
 'co-chairman',
 'lawer',
 'relation, vice chairman',
 'chairman, ceo',
 'employee',
 "head of microsoft's application software group",
 'chariman',
 'exectuitve director',
 'vice chairman',
 'shareholder',
 'relation and ceo',
 'founder, chairwoman, ceo',
 'founder and execu

In [63]:
df['company.relationship'] = df['company.relationship'].apply(
    lambda x: str(x).lower().replace('/', ', ')
)

### name

Listing possible inconsistancies where the names are different, but the following features are the same:
<ul>
    <li>company.founded</li>
    <li>company.name</li>
    <li>company.relationship</li>
    <li>company.sector</li>
    <li>company.type</li>
    <li>demographics.gender</li>
    <li>location.citizenship</li>
    <li>location.country code</li>
    <li>location.region</li>
    <li>wealth.type</li>
    <li>wealth.how.category</li>
    <li>wealth.how.from emerging</li>
    <li>wealth.how.industry</li>
    <li>wealth.how.inherited</li>
</ul>

In [64]:
dup_cols = ['company.founded',
            'company.name',
            'company.relationship',
            'company.sector',
            'company.type',
            'demographics.gender',
            'location.citizenship',
            'location.country code',
            'location.region',
            'wealth.type',
            'wealth.how.category',
            'wealth.how.from emerging',
            'wealth.how.industry',
            'wealth.how.inherited']

dup_rows = df[df.duplicated(subset=dup_cols)]
dup_rows[duprows[]]

SyntaxError: invalid syntax (1649346416.py, line 17)

In [ ]:
import re

df[df['name'].apply(lambda x: (re.search('abraham', x, re.IGNORECASE) is not None))]

In [ ]:
import re

df[df['name'].apply(lambda x: (re.search('abraham', x, re.IGNORECASE) is not None))]